In [1]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torch torchvision torchaudio


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install facenet-pytorch

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install git+https://github.com/jacobgil/pytorch-grad-cam.git


  Cloning https://github.com/jacobgil/pytorch-grad-cam.git to c:\users\dell\appdata\local\temp\pip-req-build-ibv03vb7
  Resolved https://github.com/jacobgil/pytorch-grad-cam.git to commit 1ff3f58818baa2889f3f51d0b9759783b4333ba0
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for grad-cam: filename=grad_cam-1.5.2-py3-none-any.whl size=38041 sha256=af9c774796cda15f720666c3bc4f26491ab50901adb3ddcee3ae10f6b2c24b9d
  Stored in directory: C:\Users\DELL\AppData\Local\Temp\pip-ephem-wheel-cache-if1dlex8\wheels\a0\4d\c8\0502c44e32030c99ffab1b98075308a1ef9829c1835537afc6
Successfully built grad-cam
  Attempting uninstall: grad-cam
    Found existing installation: grad-cam 1.5.0
    Uninstalling grad-ca

  Running command git clone --filter=blob:none --quiet https://github.com/jacobgil/pytorch-grad-cam.git 'C:\Users\DELL\AppData\Local\Temp\pip-req-build-ibv03vb7'


In [6]:
import gradio as gr
import torch
import torch.nn.functional as F
from facenet_pytorch import MTCNN, InceptionResnetV1
import numpy as np
from PIL import Image
import cv2
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import warnings
warnings.filterwarnings("ignore")

In [7]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

mtcnn = MTCNN(
    select_largest=False,
    post_process=False,
    device=DEVICE
).to(DEVICE).eval()

In [8]:
model = InceptionResnetV1(
    pretrained="vggface2",
    classify=True,
    num_classes=1,
    device=DEVICE
)



In [9]:
checkpoint = torch.load("resnetinceptionv1_epoch_32.pth", map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
model.eval()

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [11]:
def predict(input_image:Image.Image):
    """Predict the label of the input_image"""
    face = mtcnn(input_image)
    if face is None:
        raise Exception('No face detected')
    face = face.unsqueeze(0) # add the batch dimension
    face = F.interpolate(face, size=(256, 256), mode='bilinear', align_corners=False)
    
    # convert the face into a numpy array to be able to plot it
    prev_face = face.squeeze(0).permute(1, 2, 0).cpu().detach().int().numpy()
    prev_face = prev_face.astype('uint8')

    face = face.to(DEVICE)
    face = face.to(torch.float32)
    face = face / 255.0
    face_image_to_plot = face.squeeze(0).permute(1, 2, 0).cpu().detach().int().numpy()

    target_layers=[model.block8.branch1[-1]]
    use_cuda = True if torch.cuda.is_available() else False
    cam = GradCAM(model=model, target_layers=target_layers, use_cuda=use_cuda)
    targets = [ClassifierOutputTarget(0)]

    grayscale_cam = cam(input_tensor=face, targets=targets, eigen_smooth=True)
    grayscale_cam = grayscale_cam[0, :]
    visualization = show_cam_on_image(face_image_to_plot, grayscale_cam, use_rgb=True)
    face_with_mask = cv2.addWeighted(prev_face, 1, visualization, 0.5, 0)

    with torch.no_grad():
        output = torch.sigmoid(model(face).squeeze(0))
        prediction = "real" if output.item() < 0.5 else "fake"
        
        real_prediction = 1 - output.item()
        fake_prediction = output.item()
        
        confidences = {
            'real': real_prediction,
            'fake': fake_prediction
        }
    return confidences, face_with_mask


In [12]:
import gradio as gr

interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(label="Input Image", type="pil"),
    outputs=[
        gr.Label(label="Class"),
        gr.Image(label="Face with Explainability", type="pil")
    ],
).launch(share=True)

Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 4.26.0, however version 4.29.0 is available, please upgrade.
--------


OSError: [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software: 'C:\\Users\\DELL\\anaconda3\\Lib\\site-packages\\gradio\\frpc_windows_amd64_v0.2'

Exception ignored in: <function BaseCAM.__del__ at 0x000001E5F3BD3E20>
Traceback (most recent call last):
  File "C:\Users\DELL\anaconda3\Lib\site-packages\pytorch_grad_cam\base_cam.py", line 189, in __del__
    self.activations_and_grads.release()
    ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'GradCAM' object has no attribute 'activations_and_grads'
Traceback (most recent call last):
  File "C:\Users\DELL\anaconda3\Lib\site-packages\gradio\queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL\anaconda3\Lib\site-packages\gradio\route_utils.py", line 261, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL\anaconda3\Lib\site-packages\gradio\blocks.py", line 1786, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL\anaconda3\Lib

In [13]:
pip install --upgrade pydantic


     ---------------------------------------- 0.0/125.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/125.2 kB ? eta -:--:--
     --- ------------------------------------ 10.2/125.2 kB ? eta -:--:--
     ------------ ------------------------ 41.0/125.2 kB 326.8 kB/s eta 0:00:01
     ------------------------ ------------ 81.9/125.2 kB 508.4 kB/s eta 0:00:01
     -----------------------------------  122.9/125.2 kB 654.9 kB/s eta 0:00:01
     ------------------------------------ 125.2/125.2 kB 563.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/423.9 kB ? eta -:--:--
   ------ --------------------------------- 71.7/423.9 kB 2.0 MB/s eta 0:00:01
   ----------- ---------------------------- 122.9/423.9 kB 1.8 MB/s eta 0:00:01
   --------------------- ------------------ 225.3/423.9 kB 1.7 MB/s eta 0:00:01
   --------------------------- ------------ 286.7/423.9 kB 1.6 MB/s eta 0:00:01
   ------------------------------------- -- 399.4/423.9 kB 1.8 MB/s et

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.4 requires pydantic<2.0, but you have pydantic 2.8.2 which is incompatible.


In [14]:
import torch

# Define the current training epoch
epoch = 32  # Assuming epoch 32

# After training is complete
# Assuming 'model' is your PyTorch model object

# Specify the file path where you want to save the checkpoint file
checkpoint_path = "resnetinceptionv1_epoch_32.pth"

# Save the model's state dictionary to the checkpoint file
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    # Add any other relevant information you want to save
}, checkpoint_path)

print(f"Checkpoint saved to '{checkpoint_path}'")


Checkpoint saved to 'resnetinceptionv1_epoch_32.pth'


In [16]:
import os

file_path = "resnetinceptionv1_epoch_32.pth"
if os.path.exists(file_path):
    print(f"File '{file_path}' exists.")
else:
    print(f"File '{file_path}' does not exist.")

File 'resnetinceptionv1_epoch_32.pth' exists.
